<a href="https://colab.research.google.com/github/PeterStoreshaw/PeterStoreshaw.github.io/blob/master/Question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Question 2, What Animal? A deep learning model that can identify 10 different animal species.

To Begin with we will use the code from fast AI
'bird or not'

In [ ]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai

The following downloads duck duck go search and its upgraded version if needed.

In [ ]:
# Skip this cell if you already have duckduckgo_search installed
!pip install -Uqq duckduckgo_search

In [ ]:
!pip install --upgrade duckduckgo_search

In [ ]:
!pip install fastbook

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=200): return L(ddg_images(term, max_results=max_images)).itemgot('image')

Let's start by searching for a bird photo and seeing what kind of result we get. We'll start by getting URLs from a search:

In [ ]:
from fastbook import search_images_ddg 
urls = search_images_ddg('bird photos', max_images=1) 
urls[0]

...and then download a URL and take a look at it:

In [ ]:
from fastdownload import download_url
dest = 'bird.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

Now let's do the same with "cat photos":

In [ ]:
download_url(search_images_ddg('cat photos', max_images=1)[0], 'cat.jpg', show_progress=False) 
Image.open('cat.jpg').to_thumb(256,256) 


Now let's do the same with "gorilla photos":

In [ ]:
download_url(search_images_ddg('gorilla photos', max_images=1)[0], 'gorilla.jpg', show_progress=False) 
Image.open('gorilla.jpg').to_thumb(256,256) 

Now let's do the same with "insect photos":

In [ ]:
download_url(search_images_ddg('insect photos', max_images=1)[0], 'insect.jpg', show_progress=False) 
Image.open('insect.jpg').to_thumb(256,256) 

Now let's do the same with "fish photos":

In [ ]:
download_url(search_images_ddg('fish photos', max_images=1)[0], 'fish.jpg', show_progress=False) 
Image.open('fish.jpg').to_thumb(256,256) 

Now let's do the same with "kangaroo photos":

In [ ]:
download_url(search_images_ddg('kangaroo photos', max_images=1)[0], 'kangaroo.jpg', show_progress=False) 
Image.open('kangaroo.jpg').to_thumb(256,256) 

Now let's do the same with "koala photos":

In [ ]:
download_url(search_images_ddg('koala photos', max_images=1)[0], 'koala.jpg', show_progress=False) 
Image.open('koala.jpg').to_thumb(256,256) 

Now let's do the same with "elephant photos":

In [ ]:
download_url(search_images_ddg('elephant photos', max_images=1)[0], 'elephant.jpg', show_progress=False) 
Image.open('elephant.jpg').to_thumb(256,256) 

Now let's do the same with "lizards photos":

In [ ]:
download_url(search_images_ddg('lizard photos', max_images=1)[0], 'lizard.jpg', show_progress=False) 
Image.open('lizard.jpg').to_thumb(256,256) 

Now let's do the same with "frog photos":

In [ ]:
download_url(search_images_ddg('frog photos', max_images=1)[0], 'frog.jpg', show_progress=False) 
Image.open('frog.jpg').to_thumb(256,256) 

Our searches seem to be giving reasonable results, so let's grab 200 examples of each of "bird" and "forest" photos, and save each group of photos to a different folder:

If using google colab we can just grab those photos already downloaded.
If runing for the first time skip this step 

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

!unzip '/content/drive/MyDrive/Q2t.zip'

path = Path('Q2') 

In [ ]:
searches = 'cat', 'gorilla', 'insect', 'fish', 'koala', 'kangaroo', 'lizard', 'frog', 'elephant', 'bird' 
path = Path('Q2') 
from time import sleep 

for o in searches: 
    dest = (path/o) 
    dest.mkdir(exist_ok=True, parents=True) 
    download_images(dest, urls=search_images_ddg(f'{o} photo')) 
    sleep(10)  # Pause between searches to avoid over-loading server 
    download_images(dest, urls=search_images_ddg(f'{o} sun photo')) 
    sleep(10) 
    download_images(dest, urls=search_images_ddg(f'{o} shade photo')) 
    sleep(10) 
    resize_images(path/o, max_size=400, dest=path/o) 


## Step 2: Train our model

Some photos might not download correctly which could cause our model training to fail, so we'll remove them:

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

To train a model, we'll need `DataLoaders`, which is an object that contains a *training set* (the images used to create a model) and a *validation set* (the images used to check the accuracy of a model -- not used during training). In `fastai` we can create that easily using a `DataBlock`, and view sample images from it:

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)

Here what each of the `DataBlock` parameters means:

    blocks=(ImageBlock, CategoryBlock),

The inputs to our model are images, and the outputs are categories (in this case, "bird" or "forest").

    get_items=get_image_files, 

To find all the inputs to our model, run the `get_image_files` function (which returns a list of all image files in a path).

    splitter=RandomSplitter(valid_pct=0.2, seed=42),

Split the data into training and validation sets randomly, using 20% of the data for the validation set.

    get_y=parent_label,

The labels (`y` values) is the name of the `parent` of each file (i.e. the name of the folder they're in, which will be *bird* or *forest*).

    item_tfms=[Resize(192, method='squish')]

Before training, resize each image to 192x192 pixels by "squishing" it (as opposed to cropping it).

Now we're ready to train our model. The fastest widely used computer vision model is `resnet18`. You can train this in a few minutes, even on a CPU! (On a GPU, it generally takes under 10 seconds...)

`fastai` comes with a helpful `fine_tune()` method which automatically uses best practices for fine tuning a pre-trained model, so we'll use that.

If running in colab, once all photos downloaded. Run this box, otherwise skip.

In [ ]:
from google.colab import drive

# mount will create a folder at the left folder section on the page, but it will take a minute or so to see it

drive.mount('/content/drive') 

!zip -r Q2.zip Q2

!cp Q2.zip /content/drive/My\ Drive/ 

Run the learner

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(3)

Create and plot a confusion matrix

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

Show the worst 8 photos for confusion

In [ ]:
interp.plot_top_losses(8, nrows=1)


## Step 3: Use our model (and build your own!)

Let's see what our model thinks about that bird we downloaded at the start:

In [ ]:
guess,_,probs = learn.predict(PILImage.create('images.jpg'))
print(f"This is a: {guess}.")
